# Step 1: IMPORT THE LIBRARIES

In [1]:
import re
import gensim
import numpy as np
import pandas as pd

from konlpy.tag import Okt
from gensim import corpora
from pymongo import MongoClient
from bson.objectid import ObjectId

ImportError: DLL load failed while importing _jpype: DLL 초기화 루틴을 실행할 수 없습니다.

# Step 2: CONNECTION TO THE DATABASE

In [ ]:
client = MongoClient('mongodb://203.255.92.141:27017', authSource='admin')
db = client['SCIENCEON']
db.list_collection_names()
scienceOn_author = db['Author']
scienceOn_authorPapers = db['AuthorPapers']
scienceOn_rawData = db['Rawdata']

In [ ]:
import tweepy
print(tweepy.__version__)

3.10.0


# Step 3: FIND THE AUTHOR RESEARCH ID

In [ ]:
author_cursor = scienceOn_author.find({'name':'유재수', 'inst': '충북대학교'})
for author in author_cursor:
    researcher_ID = author['_id']
print(researcher_ID)

s101957


# Step 4: FIND ALL THE PAPERS OF THE AUTHOR HAVING THE RESEARCH ID WE FOUND & PUT IT IN A DATAFRAME

In [ ]:
dfPapers = pd.DataFrame(columns=['papers'])

In [ ]:
authorPapers_cursor = scienceOn_authorPapers.find({'A_ID':researcher_ID})
for authorPapers in authorPapers_cursor:
    papers = authorPapers['papers']
    for i in range(len(papers)):
        papersID = papers[i]
        objInstance = ObjectId(papersID)
        rawData_cursor = scienceOn_rawData.find({ "_id" : objInstance })
        for document in rawData_cursor:
            if type(document['paper_keyword']) != list:
                new_document = document['title'] + ' ' + document['english_title'] + ' ' + document['abstract'] + ' ' + document['paper_keyword'] + ' ' + document['english_abstract']
            else:
                paper_keyword = ''
                for j in range(len(document['paper_keyword'])):
                    paper_keyword += document['paper_keyword'][j] + ' '
                new_document = document['title'] + ' ' + document['english_title'] + ' ' + document['abstract'] + paper_keyword + document['english_abstract']
            df_new_document = pd.DataFrame(data=np.array([[new_document]]), columns=['papers'])
            dfPapers = pd.concat([dfPapers,df_new_document], ignore_index=True)
documents = dfPapers

# Step 5: PRINT THE DATAFRAME

Data Cleaning

In [ ]:
# Remove punctuation
documents['papers'] = documents['papers'].map(lambda x: re.sub(r'[^\w\s]',' ',x))

# Convert the titles to lowercase
documents['papers'] = documents['papers'].map(lambda x: x.lower())

# Print out the first rows of papers
documents['papers'].head()

0    hdfs에서 적응형 캐시 관리 기법 adaptive cache management ...
1    이미지 데이터 마이닝을 이용한 모바일 기반 금형 검색 시스템 a mold searc...
2    빅데이터 활성화 정책 및 응용 사례    다양한 정보 채널의 등장과 함께 빅데이터에...
3                               빅데이터 병렬 처리 기술 동향      
4                                 4차 산업혁명에서 빅데이터      
Name: papers, dtype: object

In [ ]:
documents

,papers
0,hdfs에서 적응형 캐시 관리 기법 adaptive cache management ...
1,이미지 데이터 마이닝을 이용한 모바일 기반 금형 검색 시스템 a mold searc...
2,빅데이터 활성화 정책 및 응용 사례 다양한 정보 채널의 등장과 함께 빅데이터에...
3,빅데이터 병렬 처리 기술 동향
4,4차 산업혁명에서 빅데이터
5,화장품 추천을 위한 개인의 피부 유형 및 유전자를 이용한 빅데이터 분석 기반 모바일...
6,nosql 데이터베이스 엔진을 이용한 스토리지 벤치마킹 시스템 storage ben...
7,노화 관련 유전자의 후성유전학적 접근 epigenomic approaches for...
8,이미지 데이터 마이닝을 이용한 모바일 기반 금형 검색 시스템 a mold searc...
9,ycsb 기반의 데이터베이스 엔진 벤치마킹 gui 설계 design of gui f...


# LDA PART

# Step 1: Data Cleaning & Prepare text for LDA analysis

DataFrame to list

In [ ]:
list_of_documents = list(documents['papers'])
list_of_documents[0]

'hdfs에서 적응형 캐시 관리 기법 adaptive cache management scheme in hdfs 스마트팩토리는 정보통신기술 ict 를 이용한 공정의 모든 데이터를 수집  분석하여 제어하고 있다  기존보다 방대한 양의 데이터를 처리하기 위해 기업들은 하둡을 이용한다  다양한 크기의 데이터가 나타나는 환경에서 hdfs을 효율적으로 관리하기 위한 적응형 캐시 관리 기법을 제안한다  제안하는 기법은 데이터 노드의 로컬 디스크의 공간 이용 효율성을 높이고 평균 데이터 크기를 분석하여 데이터 노드 확장시 적합한 블록 크기를 적용할 수 있게 관리한다  성능 평가를 통해 제안하는 기법의 데이터 노드에서 로컬 디스크 효율 향상과 읽기와 쓰기 속도의 속도에 효과를 보인다 '

Tokenize

In [ ]:
t = Okt()
pos = lambda d: ['/'.join(p) for p in t.pos(d, stem=True, norm=True)] #t.pos(d, stem=True, norm=True) or t.nouns(d)
texts_ko = [pos(doc) for doc in list_of_documents]
print(texts_ko[0])

SystemError: java.lang.UnsatisfiedLinkError: Can't load library: C:\Users\admin\AppData\Local\Programs\Python\Python39\lib\site-packages\_jpype.cp39-win_amd64.pyd

In [ ]:
import sys
sys.version

'3.9.6 (tags/v3.9.6:db3ff76, Jun 28 2021, 15:26:21) [MSC v.1929 64 bit (AMD64)]'

Encode tokens to integers

In [ ]:
dictionary_ko = corpora.Dictionary(texts_ko)
dictionary_ko.save('ko.dict')  # save dictionary to file for future use

NameError: name 'texts_ko' is not defined

Calculate TF-IDF

In [ ]:
#from gensim import models
tf_ko = [dictionary_ko.doc2bow(text) for text in texts_ko]
tfidf_model_ko = gensim.models.TfidfModel(tf_ko)
tfidf_ko = tfidf_model_ko[tf_ko]
corpora.MmCorpus.serialize('ko.mm', tfidf_ko) # save corpus to file for future use

# print first 10 elements of first document's tf-idf vector
print(tfidf_ko.corpus[0][:10])
# print top 10 elements of first document's tf-idf vector
print(sorted(tfidf_ko.corpus[0], key=lambda x: x[1], reverse=True)[:10])
# print token of most frequent element
print(dictionary_ko.get(27))

NameError: name 'texts_ko' is not defined

# Step 3: LDA model training

In [ ]:
lda_model = gensim.models.LdaMulticore(corpus=tfidf_ko,
                                       id2word=dictionary_ko,
                                       num_topics=5)

In [ ]:
keywords = lda_model.print_topics(-1,5)
keywords

[(0,
  '0.007*"병렬/Noun" + 0.007*"동향/Noun" + 0.006*"처리/Noun" + 0.005*"k/Alpha" + 0.005*"top/Alpha"'),
 (1,
  '0.018*"지/Josa" + 0.018*"스토리/Noun" + 0.011*"벤치마킹/Noun" + 0.011*"nosql/Alpha" + 0.007*"엔진/Noun"'),
 (2,
  '0.006*"피부/Noun" + 0.005*"금/Noun" + 0.004*"벤치마킹/Noun" + 0.004*"형/Suffix" + 0.004*"gui/Alpha"'),
 (3,
  '0.007*"4/Number" + 0.006*"차/Noun" + 0.006*"산업혁명/Noun" + 0.006*"패턴/Noun" + 0.006*"피부/Noun"'),
 (4,
  '0.008*"색인/Noun" + 0.007*"그래프/Noun" + 0.005*"패턴/Noun" + 0.005*"유전자/Noun" + 0.005*"요약/Noun"')]

# KEYWORD PART

In [ ]:
keywords = []
for topic in lda_model.print_topics(-1,5):
    topic_list = topic[1].split('+')
    for i in range(len(topic_list)):
        count = 0
        words = topic_list[i].split('"')
        for j in range(len(words)):
            if "*" in words[j] or words[j] == "" or words[j] == " ":
                continue
            elif words[j] not in keywords:
                word = words[j].split('/')
                if word[0] not in keywords and word[1] == "Noun":
                    count += 1
                    keywords.append(word[0])
                    break
        if count >= 1:
            break

In [ ]:
print(keywords)

['상품', '피부', '색인', '병렬', '스토리']
